<a href="https://colab.research.google.com/github/antonpolishko/task-ties/blob/master/ExtractiveSummarizer_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Set up

In [0]:
# !pip install bert-extractive-summarizer -q
# !pip install spacy==2.1.3 -q
!pip install transformers -q #==2.2.2
# !pip install neuralcoref -q
# !pip install ipywidgets -q
import pandas as pd
import numpy as np
import os
from pathlib import Path, PurePath
from transformers import *
# from summarizer import Summarizer
import logging
import torch
from numpy import ndarray
from typing import List
from tqdm.notebook import tqdm
# import spacy


     |████████████████████████████████| 573kB 1.9MB/s 
     |████████████████████████████████| 1.0MB 51.2MB/s 
     |████████████████████████████████| 3.7MB 31.1MB/s 
     |████████████████████████████████| 870kB 53.1MB/s 


In [0]:
# set data paths, this requires local drive to have a folder calld "COVID-19" with the metadata.csv file
# returns a string to the local path setup
def setup_local_data():
  from google.colab import drive
  drive.mount('/content/drive')
  drive_path=PurePath('/content/drive/My Drive')
  input_dir = drive_path/'COVID-19'
  print(list(Path(input_dir).glob('*')))
  return input_dir

In [0]:
#read the metadata file into df
def read_metadata_csv(input_dir):
    metadata_path = input_dir/ 'clean_metadata.csv'
    metadata = pd.read_csv(metadata_path, 
                         dtype={'title':str,
                                'abstract':str})
    #set the abstract to the paper title if it is null
    metadata['abstract'] = metadata['abstract'].fillna(metadata['title'])
    #remove if abstract is empty or contains only one word
    metadata = metadata.dropna(subset=['abstract'], axis = 0)
    metadata['number_tokens'] = metadata['abstract'].apply(lambda x: len(x.split()))
    metadata = metadata[metadata['number_tokens']>1].reset_index(drop=True)
    metadata = metadata.drop('Unnamed: 0', axis=1)
    return metadata

In [0]:
# read preprocessed SciBERT embeddings
def read_summary_data(input_dir):
  summary_path = input_dir/'AbstractSummaries' 
  summaries = pd.concat([pd.read_json(f) for f in Path(summary_path).glob('*')]).reset_index(drop=True)
  return summaries

In [0]:
def create_custom_model_tokenizer(pretrain_model):
    # Load model, model config and tokenizer via Transformers
    custom_config = AutoConfig.from_pretrained(pretrain_model)
    custom_config.output_hidden_states=True
    custom_tokenizer = AutoTokenizer.from_pretrained(pretrain_model)
    custom_model = AutoModel.from_pretrained(pretrain_model, config=custom_config)
    return custom_model, custom_tokenizer


In [0]:
# from spacy.lang.en import English

# class SentenceHandler(object):

#     def __init__(self, language = English):
#         self.nlp = language()
#         self.nlp.add_pipe(self.nlp.create_pipe('sentencizer'))

#     def process(self, body: str, min_length: int = 40, max_length: int = 600):
#         """
#         Processes the content sentences.
#         :param body: The raw string body to process
#         :param min_length: Minimum length that the sentences must be
#         :param max_length: Max length that the sentences mus fall under
#         :return: Returns a list of sentences.
#         """
#         doc = self.nlp(body)
#         doc.is_parsed=True
#         return [c.string.strip() for c in doc.sents if max_length > len(c.string.strip()) > min_length]

#     def __call__(self, body: str, min_length: int = 40, max_length: int = 600):
#         return self.process(body, min_length, max_length)

In [0]:
def extract_summary(text, custom_model=None, custom_tokenizer = None):
    model = Summarizer(custom_model=custom_model, custom_tokenizer=custom_tokenizer)
    return model(text)

#Main code

In [0]:
local_dir = setup_local_data()
metadata = read_metadata_csv(local_dir)
metadata.info()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[PosixPath('/content/drive/My Drive/COVID-19/TestSearchResults_Incubation.csv'), PosixPath('/content/drive/My Drive/COVID-19/TestSearchResults_Incubation.gsheet'), PosixPath('/content/drive/My Drive/COVID-19/TestSearchResults_Incubation_v5.csv'), PosixPath('/content/drive/My Drive/COVID-19/TestSearchResults_Incubation_v5_target.csv'), PosixPath('/content/drive/My Drive/COVID-19/CORD-19-research-challenge-v5.zip'), PosixPath('/content/drive/My Drive/COVID-19/data'), PosixPath('/content/drive/My Drive/COVID-19/covid_vectors_part_0.json'), PosixPath('/content/drive/My Drive/COVID-19/covid_vectors_part_1.json'), PosixPath('/content/drive/My Drive/COVID-19/covid_vectors_part_2.json'), PosixPath('/content/drive/My Drive/COVID-19/covid_vectors_part_3.json'), PosixPath('/content/drive/My Drive/COVID-19/covid_vectors_part_4.json'), PosixPath('/content/drive/My Drive/C

In [0]:
#instantiate custom model and tokenizer
sciBert, sciBert_tokenizer = create_custom_model_tokenizer('allenai/scibert_scivocab_uncased')

NameError: ignored

In [0]:
CHUNKS_COUNT = 20

In [0]:
#extract summaries from the abstracts
#chunk processing due to long processing time and possible notebook runtime shutdowns
chunks = np.array_split(metadata, CHUNKS_COUNT)
# chunk_dfs = []
for index in range(3,len(chunks)):
    chunk = chunks[index].reset_index()
    summary_list = []
    for i in tqdm(chunk.index):
        summary = extract_summary(chunk.iloc[i]['abstract'], 
                                  custom_model=sciBert, 
                                  custom_tokenizer=sciBert_tokenizer)
        summary_list.append(
        {
        "cord_uid": chunk.iloc[i]['cord_uid'],
        "sha": chunk.iloc[i]['sha'],
        # "scibert_emb": bc.encode([summary])[0],
        "summary": summary
        })
    summary_df = pd.DataFrame(data=summary_list)
    summary_df.to_json(local_dir/'abstract_summaries_part{}.json'.format(index))


In [0]:
!pip install --upgrade git+https://github.com/zalandoresearch/flair.git -q
from flair.data import Sentence
from flair.embeddings import BertEmbeddings,DocumentPoolEmbeddings
import glob


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 983kB 3.9MB/s 
     |████████████████████████████████| 798kB 21.7MB/s 
     |████████████████████████████████| 256kB 21.4MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.


In [0]:
emb_model = BertEmbeddings(bert_model_or_path="allenai/scibert_scivocab_uncased", 
                           layers='-2')
def get_embeddings(text, model):
  sentence = Sentence(text)
  document_embedding = DocumentPoolEmbeddings([model],
                                             pooling= 'mean')
  document_embedding.embed(sentence)
  # now check out the embedded sentence.
  return sentence.get_embedding().data.numpy()

In [0]:
CHUNKS_COUNT=3
files = glob.glob('/content/drive/My Drive/COVID-19/AbstractSummaries/*.json')

for index in range(len(files)):
    df = pd.read_json(files[index]).dropna(subset=['summary']).reset_index(drop=True)

    chunks = np.array_split(df, CHUNKS_COUNT)
    
    for chunk_idx in range(len(chunks)):
      chunk = chunks[chunk_idx].reset_index()
      emb_list=[]
      for i in tqdm(chunk.index):
        try:
          embedding = get_embeddings(chunk.iloc[i]['summary'], emb_model)
          emb_list.append(
          {
            "cord_uid": chunk.iloc[i]['cord_uid'],
            "sha": chunk.iloc[i]['sha'],
            "scibert_emb": embedding
          })
        except RuntimeError:
          emb_list.append(
          {
            "cord_uid": chunk.iloc[i]['cord_uid'],
            "sha": chunk.iloc[i]['sha'],
            "scibert_emb": np.nan
          })
      emb_df = pd.DataFrame(data=emb_list)
      emb_df.to_json(local_dir/'AbstractEmbeddings'/'abstract_embeddings_part{}_{}.json'.format(index, chunk_idx),
                     default_handler=str)
      del emb_df



2020-04-06 17:00:59,295 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 17:01:07,185 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 17:01:28,256 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 17:01:29,027 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 17:01:33,183 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 17:01:35,828 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 17:01:38,158 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 17:01:40,372 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 17:01:52,133 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 17:01:58,205 ACHTUNG: An empty Sentence was 

2020-04-06 17:03:05,321 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 17:03:33,115 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 17:03:35,250 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 17:03:53,394 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 17:04:01,117 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 17:04:16,843 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 17:04:17,519 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 17:04:19,271 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?



2020-04-06 17:04:46,638 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 17:06:13,077 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 17:06:16,590 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?



2020-04-06 17:06:34,321 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 17:06:44,520 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 17:07:41,370 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?



2020-04-06 17:08:41,856 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 17:08:42,008 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 17:08:44,026 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 17:08:59,351 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 17:08:59,367 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 17:09:00,454 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 17:09:01,180 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 17:09:03,070 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 17:09:05,067 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 17:09:07,827 ACHTUNG: An empty Sentence was 

...

2020-04-06 18:32:41,676 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 18:32:48,370 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 18:32:48,387 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 18:33:00,311 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 18:33:05,999 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 18:33:10,876 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 18:33:16,885 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 18:33:19,205 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 18:33:23,617 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 18:33:24,681 ACHTUNG: An empty Sentence was 

2020-04-06 18:34:21,574 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 18:34:31,303 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 18:34:36,055 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 18:34:38,819 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 18:34:44,225 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 18:34:59,758 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 18:35:17,358 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 18:35:17,373 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 18:35:22,351 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 18:35:57,739 ACHTUNG: An empty Sentence was 

2020-04-06 18:36:21,232 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 18:36:26,006 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 18:36:32,553 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 18:36:33,221 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 18:36:39,276 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 18:37:17,838 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 18:37:34,778 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 18:37:42,131 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?



2020-04-06 18:38:51,187 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 18:39:15,161 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 18:39:17,748 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?



2020-04-06 18:39:55,814 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 18:40:11,600 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-04-06 18:41:09,556 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?

